<div>
    <table>
        <tr>
            <td>
                    <h1>Brightway2 seminar</h1>
                    Chris Mutel <a href="https://www.psi.ch/">(PSI)</a>
                    Pascal Lesage <a href="http://www.ciraig.org/en/">(CIRAIG)</a>
            </td>
        </tr>
    </div>

# Brightway modules: bw2io, bw2data, bw2calc

*The following content is taken and adapted from [the course materials dispensed at the 2022 Brightcon conference](https://github.com/Depart-de-Sentier/teaching-material/tree/main/beginners/Projects%2C%20databases%2C%20exchanges%2C%20activities). If you need additional notebooks, go to this repository*.


<div class="alert alert-info">
Note: we will be using Brightway2, not Brightway25. From the user end side, very little differ between the two.
</div>





**Brightway2 documentation** 👉 [https://docs.brightway.dev/en/legacy/index.html](https://docs.brightway.dev/en/legacy/index.html)

**Brightway25 documentation** 👉 [https://docs.brightway.dev/en/latest/index.html](https://docs.brightway.dev/en/latest/index.html)

### Learning objectives  
  - Learn about the general structure of Brightway and its most importand abstractions: projects, databases, activities and exchanges  
  - Learn how to find objects (notably activities and exchanges), assign them to variables and work with them using their associated methods  
  - Learn about how simple LCA calculations are done (one product, one impact category), and specifically how the different matrices are built and used  
  - Learn how to extract information from the matrices (inputs or results) and translate them into nice, human-readable objects  
  - Learn different ways to carry out comparative LCAs  
  - Learn different ways to carry out LCAs with multiple impact categories

### Content  

#### 1) Getting started  
##### 1.1) Jupyter lab / notebooks and Brightway2  
  - A quick tour of jupyter lab / notebooks  
  - Accessing Brightway2 libraries  
  
##### 1.2) Projects  
  - Concept  
  - Creating a new project, or switching to an existing project  
  - Contents of a project  

  
##### 1.3) bw2_setup(), biosphere3 database and LCIA methods  
  - bw2_setup()  
  - biosphere3 database and a first look at database objects  
  - Getting activities from codes or keys  
  - Methods  
  - Looking up elementary flows (list comprehensions, search)  
  - Searching for methods  
  - Nice display of data in methods 

##### 1.4) LCI databases  
  - Importing (succinct)  
  - LCI activities  
  - Looking up activities  
  - LCI exchanges
  - Loaded LCI databases
  
#### 2) My first LCA - simplest case:  
##### 2.1) General syntax of LCA calculations  

##### 2.2) The `demand` attribute  

##### 2.3) Reminder of the system that needs to be solved in calculating an LCI  

##### 2.4) Building the matrices  

  - $A$ matrix  
  - $B$ matrix  
  - $f$ (demand array)  
  
##### 2.5) Solution to the inventory calculation  

  - Supply array  
  - Inventory matrix  
  
##### 2.6) Life Cycle Impact Assessment  

  - `.lcia()` method  
  - Simple contribution analysis  
  
#### 3) My second LCA - comparative LCA:
    
#### 4) My third LCA - Multiple impact categories
  
#### 5) My first and third LCAs revisited with MultiLCA

### 1) Getting started

#### 1.1) Jupyter lab / notebooks and accessing Brightway2

This teaching material was produced with the intention to be used in Jupyter Notebooks.

For installation instructions, consult the [brightway documentation](https://docs.brightway.dev/en/legacy/content/installation/installation.html)

<div class="alert alert-info">
This notebook is meant to run with a specific conda (and python) environment.
    Make sure you have started Jupyter Notebooks or Jupyter Lab from the conda
    environment where the brightway2 modules are installed.
</div>

##### Accessing Brightway2 libraries

The different modules in Brightway2 are Python libraries. This means that, to use them, you can use any environment from which you normally use Python (Idle, command prompt, Spyder or, as is the case today, Jupyter Notebooks).  

We will favour Jupyter Notebooks in this course because it allows us to integrate code and text. It will also allow us to provide code snippets for you to complete.  

Like all other Python packages, you need to `import` Brightway2 modules. We will here import it as `bw`. This means that, to access a **method** or **function** from the brightway2 modules, the prefix `bw.` needs to be added. 

In [ ]:
import brightway2 as bw

Alternatively, we can import the Brightway's sub-modules directly:

In [ ]:
import bw2io, bw2data, bw2calc

We're also going to be using the following libraries:

In [ ]:
import os               # to use "operating system dependent functionality"
import numpy as np      # "the fundamental package for scientific computing with Python"
import pandas as pd     # "high-performance, easy-to-use data structures and data analysis tools" for Python

#### 1.2) Projects

##### Concept

The top-level containent in Brightway2 is the project (see [here](https://2.docs.brightway.dev/intro.html#projects) for a description and [here](https://2.docs.brightway.dev/technical/bw2data.html#projects) for the **technical docs about projects**). A project contains LCI databases, LCIA methods and other less often used objects. Objects from one project do not interract with objects within other projects. By analogy, projects are like databases in other LCA software.  
When you first launch Brightway2, you will be in the `default` project. You can check this using the `current` property of the `projects` object: 

##### Creating a new project, or switching to an existing project

Let's create a new project for this seminar, unsurprisingly called "bw2_seminar_2017". There are two ways of doing this:  
* `projects.create_project('brightcon22')` will create the project, but you will remain in your current project.
* `projects.set_current('brightcon22')` will switch you to the project passed as argument, and create it first if it doesn't exist.  Let's do the latter:

In [ ]:
# The name of the project is entered as string; 
# it doesn't really have any restrictions, so can include spaces, 
# special characters, other languages, or even emoji.



You can always see what projects you have on your computer by running `list(bw.projects)`. Unless you have worked with Brightway2 before on your computer, your list should contain two projects: 'default' and 'bw2_seminar_2017'.

_**Exercise**_: list the projects on your computer.

Like in all Python modules, you can get additionnal information on the `projects` object and associated properties and methods by typing `help(projects)`. The [docs](https://docs.brightwaylca.org/technical/bw2data.html#projects) are of course more verbose.

##### Contents of a project

One property of `projects` is its location, given by `projects.dir`:

<div class="alert alert-info">
    Try to locate your project directory on your computer.
</div>

First things first: **do not panic**! You can use Brightway2 for years without ever opening this directory, but we will discuss some of these files later.

It is simply interesting to note that, for now, all the directories are empty except the `lci` directory, which contains an empty database.

All in all, the project takes up about 150KB.  
Its now time to start populating the project.

#### 1.3) bw2setup(), biosphere3 database and LCIA methods

##### bw2setup()

The first thing you should do is add flow and LCIA methods. This is done by running the `bw2setup` function:

The output tells us that bw2_setup created some very useful things:  
  - Created a database called "biosphere3": this database contains elementary flows (called biosphere exchanges in Brightway2)  
  - **762** impact assessment methods  (with bw2data version 3.6.6)

<div class="alert alert-info">
    Note: The biosphere database contains all "natural" flows human-made activities connect to. E.g., CO2 emissions, ore in ground, etc.
</div>
  
It also created some a `mapping` between the imported exchanges and some integer: more on this later.  
The whole directory now takes up 157MB.

##### biosphere3 database and a first look at database objects

<div class="alert alert-info">
    Note: You can always list the databases inside a project by simply typing 'bw.databases'. This accesses the 'database.json' file in your 'project.dir' (I learned the latter by typing `databases?`, you should try it too.)
</div>



While not impossible to interact with the data at this level, you probably never will unless you are developping some funky program. Instead, it is strongly recommended to learn to work with `abstractions`.  

To access a database in Brightway, you use the `Database` initialization method (again, you can type `Database?` for more information - this is the last time I'll mention this.

It doesn't actually return anything other than information about the Backend.  
However, there are many properties and functions associated with this database object.  These are found [here](https://2.docs.brightway.dev/technical/bw2data.html#databasechooser). We can also have a look through the autocomplete. Let's assign the database to a variable:

Let's check the my_bio `type`:

Let's check its length:

This is exactly the number of items we saw had been added to databases.db  
Given this, what do you think is going on?

If you type `my_bio.` and click on tab, you should get a list of properties and methods associated with database objects. Try this now:

In [ ]:
my_bio.get        # Type my_bio. and click tab. Have a look at the different properties and objects

Some of the more basic ones we will be using now are :  
  - `random()` - returns a random activity in the database
  - `get(*valid_exchange_tuple*)` - returns an activity, but you must know the activity key
  - `load()` - loads the whole database as a dictionary.
  - `make_searchable` - allows searching of the database (by default, it is already searchable)
  - `search` - search the database  
  
Lets start with `random`:

This returns a biosphere activity, but without assigning it to a variable, there is not much we can do with it directly.  

Note: It may seem counter-intuitive for elementary *flows* to be considered *activities* in Brightway, but it is no mistake. 
LCA models are made up of **nodes** (activities) that are linked by **edges** (exchanges). The biosphere activities are the nodes *outside* the technosphere. Emissions and resource extractions are modelled as exchanges between activities in the technosphere (part of the product system) and these biosphere activities.  

More on this later. 

For now, let's assign another random activity to a variable:

We can get the type of the object that was returned from the database:

The type is an **activity proxy**. Activity proxies allow us to interact with the content of the database.

In Brightway, we *almost* always work with `Activity` or `Exchange` objects. 

To see what the activity contains, we can convert it to a dictionary:

##### Getting activities from codes or keys

We can see that the activities in the biosphere3 database have unique **codes**, which we can use with the `get` function:

Activities can also be "gotten" via `get_activity`, but the argument is the activity **key**, consisting of a tuple with two elements: the database name, and the activity code.

**Exercise:** Use `bw.get_activity()` to retrieve the random biosphere activity. 

You can always find (or return) the key to an activity using the `.key` property.

##### Searching for activities

Let's say we are looking for a specific elementary flow, we can use the `search` method of the database (see [here](https://docs.brightwaylca.org/technical/bw2data.html#default-backend-databases-stored-in-a-sqlite-database) for more details on using search):

It is also possible to use "filters" to narrow searches, e.g.

The database object is also iterable, allowing "home-made" searches through list comprehensions. This approach is better because one can add as many criteria as wanted:

Activities returned by searches or list comprehensions can be assigned to variables, but to do so, one needs to identify the activity by index. Based on the above, I can refine my filters to ensure the list comprehension only returns one activity, and then choose it without fear of choosing the wrong one..

**Exercise**: look for and assign to a variable an emission of nitrous oxide emitted to air in the "urban air" subcompartment.

Let's leave the biosphere database here for now.

##### Methods

bw2_setup() also installed LCIA methods.

One can load a random method:

Here, the random method returns the tuple by which the method is identified. To get to an actual method, the following syntax is used:

Of course, a random method is probably not useful except to play around. To find an actual method, one can again use list comprehensions. Let's say I am interested in using the IPCC2013 100 years method:

I am interested in the second of these, and will assign it to a variable. I can will refine my search until there is one element in my list and then choose it by subscripting.

Of course, if I know exactly the method I want, and I know the syntax, I can simply type it out: `('IPCC 2021', 'climate change', 'GWP 100a')`

Again, there are a bunch of methods associated with a method object. You can access these by typing ipcc_2013_method. and clicking tab.  
For example, metadata:

Question: where is this metadata?

Let's use the `load` method to see what is in the object:

This contains tuples with (elementary flow, characterization factors).

##### Nice display of data in methods 

**Exercise:** Create a dictionary with keys = elementary flow names and values = characterization factors for the `IPCC GWP100` method (including long-term emissions).

Bonus (optional): Generate a Pandas Series with the resulting dictionary. 

Enough said for now about methods.

#### 1.4) LCI databases

Outside of the classes, you would probably use the following code to "import" ecoinvent into your brightway installation:

On a "recent" computer, this can take a few minutes to be finished.

Other code to import LCI databases in other formats are found [here](https://github.com/brightway-lca/brightway2-io/tree/master/bw2io/importers). We'll explore this later.

*We created* a new project, so let's make sure we set it as the "current" project.

To access the actual database, you need to use the Database method: 

Let's assign the database to a variable and see what we can do:

Again, we can get an idea of useful methods and attributes by typing eidb. and Tab. Do this now.

In [ ]:
eidb. #Press tab!

##### LCI activities

In the context of LCI databases, activities are the nodes "within the technosphere". They are therefore the columns in the technosphere matrix $A$.  
There are different ways to get access to an activity. Let's use the `random()` method for now to explore a random activity in the ecoinvent database.

To see what is stored in an activity object, let's convert our random act in a dictionary: 

Notice one important thing: **no exchanges**!  
Indeed, the exchanges and the activities are stored in two different tables of the `databases.db` database.  
It is possible, however, to iterate through the exchanges of the activities.

##### Searching and getting LCI activities

Searching and getting LCI activities is done exactly the same way as for activities in the biosphere3 database:

**Exercise:** Return an activity for electricity production, coal-fired power plants in Germany

#### LCI exchanges

**`Exchanges`** are the edges between nodes.

These can be:  
  - an edge between two activities within the technosphere (an element $a_{ij}$ of matrix $A$)  
  - edges between an activity in the technosphere and an activity in the "biosphere" (an element $b_{kj}$ of the biosphere matrix $B$).

One can iterate through **all** exchanges that have a given activity as `output`

One can also iterate through subsets of the exchanges:  
  - Technosphere exchanges: exchanges linking to other activities in the technosphere, `activity.technosphere()`  
  - Biosphere exchanges: AKA elementary flows, linking to activities in the biosphere database `activity.biosphere()`  
  - Production exchange: the reference flow of the activity `activity.production`  

Let's assign a **technosphere exchange** to a variable to learn more about it:

Again, the type is a proxy (refer to the diagram above about the different translation layers).

Let's now look at a production exchange

**Exercise:** Assign a biosphere flow to a variable, and check the following:  
  - Is the output the same as for the technosphere exchange?  
  - From what database does the biosphere exchange come from?  
  - What is the amount of the exchange (i.e. the weight of the edge connecting the two activities)?
  
NOTE: If you get a ` list index out of range` error when trying to subscript your list comprehension, it means your list comprehension is empty, i.e. that there are no biosphere flows associated with the activity.

#### Loaded LCI databases

It is possible to load the entire database into a dictionary.  
This greatly speeds up work if you need to iterate over all activities or exchanges. The resulting object is quite big, so you should do this only if the gain in efficiency is worth it.

## 2) My first LCA

Brightway has a so-called LCA object.  
It is instantiated using `LCA(args)`.  
The only required argument is a functional unit, described by a dictionary with keys = activities and values = amounts (more [here](https://docs.brightwaylca.org/lca.html#specifying-a-functional-unit)).  
A second argument that is often passed is an LCIA method, passed using the method tuple.  

### 2.1) General syntax of LCA calculations

Let's create our first LCA object using our random activity and our IPCC method.  

The steps to get to the impact score are as follows:

Let's not take a closer look at the LCA object and its methods/attributes. We'll do this by creating a new LCA object: 

### 2.2) the `demand` attribute

To access the actual activity from the demand, you would do this:

There are also other attributes that have simply not been built yet, such as the `demand_array` and the `score`. To generate them, we first need to actually build the matrices. This will be done when calling the `.lci()` method.

### 2.3) Reminder of the system that needs to be solved in calculating an LCI

Before actually running the `.lci()` method, here's a quick refresher of the actual calculation that Brightway will need to do to calculate the inventory:  

$g=BA^{-1}f$  

where:  

  - $A$ = the technosphere matrix  
  - $B$ = the biosphere matrix (matrix with elementary flows)  
  - $f$ = the final demand vector  
  - $g$ = the inventory  

**Discussion:** Knowing what you do about the structure of Brightway (notably, activities and exchanges), what needs to happen to generate these matrices?

To consider:  
  - how should the order of the rows and columns be determined?  
  - how should we keep track of what is in each row and column?  
  - The parameters in the matrices are sometimes actually probability distribution functions - how should we consider this uncertainty information?  
  - The matrices are *sparse*, i.e. they are mostly made up of zeros. Should we consider this? Why? How?

### 2.4) Building the matrices

#### Structured arrays

LCI data imported in Brightway is stored in the `databases.db` database, discussed above.  
It is also stored in [numpy *structured arrays*](https://docs.scipy.org/doc/numpy/user/basics.rec.html). 

**Exercise**(not core): Load the structured array of the ecoinvent database you are working with now.

In this array:  
  - `input` and `output` columns are integers that map to an activity. This mapping is found in the `mapping.pickle` file in the `project` directory and it looks something like this:

  - `row` and `col`store *dummy* placeholder information about the location of the parameter in the matrices. 
  - the `type` indicates whether the exchange is a reference flow (`type`=0), technosphere exchange (`type`=1) or elementary flow (`type`=2).  
  - the other columns deal with uncertainty data. We'll cover that later, but one can always read about these columns in the [`stats_arrays` documentation](http://stats-arrays.readthedocs.io/en/latest/)

When the `.lci()` method is called, the structured arrayas are used to build matrices. The code responsible to do this is in the [`MatrixBuilder` class methods](https://2.docs.brightway.dev/technical/bw2calc.html#matrix-builder). 

The method `MatrixBuilder.build_dictionary` is used to take input and output values, respectively, and figure out which rows and columns they correspond to. The actual code is succinct - only one line - but what it does is:
  - Get all unique values, as each value will appear multiple times
  - Sort these values
  - Give them integer indices, starting with zero
This information on row and column indices is sufficient to build matrices. These matrices are build in a [COOrdinate sparse matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html) format, where, for each exchange, three values are required: row position, column position, and amount (the actual value). The sparse matrices are actually stored in [CSR format](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix), but this is a detail.  

Some more details are are found [here](https://2.docs.brightway.dev/technical/bw2calc.html#matrix-builders).  

Let's now finally run the `.lci()`.

Here's what the structured arrays *now* look like:  

We see that the row and col numbers are no longer dummy variables, but that they actually have real matrix indices.

#### Dictionaries that map between incides and activities

One of the useful things that the `MatrixBuilder` produces are `dictionaries` that map row and column numbers to the keys of activities.  There are three such dictionaries, all directly accessible as attributes of the LCA object:
 - `activity_dict`: Columns in the technosphere matrix $A$ or biosphere matrix $B$
 - `product_dict` : Rows in the technosphere matrix $A$  
 - `biosphere_dict`: Rows in the biosphere matrix $B$

Here what this dictionary looks like:

So, if I know the key to my activity (which, again,  is a `tuple` consisting of the database name and the activity code), I can read the column index (from `activity_dict`) or row index (from `product_dict` or `biosphere_dict` for the $A$ or $B$ matrices, respectively). 

Let's find out what column is associated with the activity that is producing our final demand as reference flow.

While this is useful, it is often more useful to determine what a row or column in the matrices actually refers to. In these cases, we need a dictionary that maps row or column indices to activity keys, and not the opposite.  
We can do this by reversing our dictionaries:

As a convenience, Brightway offers a method that will generate the three reverse dictionaries simultaneously.  
`.reverse_dict()` returns three reverse dicts (reverse activity dict,  reverse product dict,  reverse biosphere dict) *in that order*. The syntax for creating and assigning these reverse dicts is therefore: 

#### $A$ and $B$ Matrices

We can also access the matrices that were constructed. Let's look at the technosphere matrix ($A$).  
The ** $A$ matrix**, with each element $a_{ij}$ provides information on the amount of input or output of product $i$ from activity $j$. When $i=j$, the element $a_{ij}$ is the *reference flow* for the activity described in the column.

I am told that the dimensions of the matrix is $n*n$ where $n$ is the number of activities in my product system, and that the amount of actually stored elements is much less than $n^2$ (because the matrix is *sparse* and zero values are not stored).  

We can have an idea of what it stores by printing it out:

It therefore stores both the coordinates and the values (as expected).
We can slice this matrix using coordinates. For example, let's say we wanted a view of the exchanges associated with the unit process providing our functional unit.  
We already know found the column number for that activity: 

To return the whole column from the $A$ matrix, we therefore slice the $A$ matrix.  
Python notes:  
  - In Python, slicing is done using []
  - We specify rows first, then columns  
  - `:` refers to "the whole row" or "the whole column" (depending if it is passed first or second in the []) 

Printing this out gives:

Not too useful: it would be better to get the *names* to these exchanges.  
We need to do two things:  
  - Get the indices from the CSR matrix (we can do this by converting it to a sparse matrix in `COOrdinate` format first)  
  - Get the activity code for the each index (we can do this using the reverse of the `activity_dict`)  
  - Use `get_activity` to access the actual names of the activities.  

1) Converting the CSR matrix to a COO matrix:  

It is still a sparse matrix with the same number of elements, and it looks quite like the CSR version when we print it out:

However, we can directly access the rows and column indices using `row` and `col`:  

2) Get the activity code for each element using the **reverse** product dictionary we produced above:

It would be even nicer to get the names for these:

We can put these in a neat Pandas Series, with actual names and amounts:

Alternative way to generate similar information without even looking at the matrices:

Note the differences:  
  - The reference flow is not there (activity.technosphere() only returns technoshere exchanges where the input is not equal to the output)  
  - The values are positive, not negative (because the $A$ matrix is $I-Z$ where $Z$ contains the information on these inputs.

**Exercise**: Create a Pandas Series with the elementary flows of the activity supplying the reference flow for myFirstLCA.

#### Demand array $f$

The demand array is the $f$ in $As=f$. 
It is an attribute of the LCA object:

Looks like it is all zeros, but not so:

So where is the one? We can know this by using our `activity_dict`.

### 2.5) Solution to the inventory calculation

We saw above how `.lci()` produced the $A$ and $B$ matrices.  
`.lci()` also *solves* the equation $As=f$ and calculated the inventory by multiplying the solution to this equation by the biosphere matrix.  

#### Supply array

Vector containing the amount each activity will need to provide to meet the functional demand, i.e. $s=A^{-1}f$.

**Inventory matrix**  
Contains the inventory *by activity* (i.e. not summed). In other words, we do not have $g=BA^{-1}f$, but rather $G=B \cdot diag(A^{-1}f)$

We can aggregate the LCI results along the columns (i.e. calculate the cradle-to-gate inventory):

**Exercise:** Get the total (cradle-to-gate) emissions of nitrous oxide emitted to air in the "urban air" subcompartment.

### 2.7) LCIA

The LCIA calculation is done via the `.lcia()` method.

A number of other matrices are now available:

Question: would there be more, less or just as many elements in the inventory matrix as there are in the characterized inventory matrix?

The overall score is now an attribute of the LCA object: 

We also could have determined what this score was by summing the elements of our `characterized_inventory` matrix:

We could also have calculated it by multiplying the inventory and characterization factors ourselves:

We could also calculate the score by elementary flow (summing columns for each rows), irrespective of the unit process that produced it:

Notice that is has **two** dimensions. The result is in fact a one-dimensional matrix:

To convert it to an array (probably more useful for many purposes), you can use any of the following approaches:

**Exercise:** Create a Pandas series that has the scores per unit process, sorted by value (contribution analysis)

## 3) My second LCA: Comparative LCA

Let's choose two activities to compare, say Swiss electricity produced from respectively a run-of-river hydropower plant and a wind turbine.

**Exercise**: assign the two activities to variables `hydro` and `wind` respectively.

Let's also compare these according to their carbon footprint as measured with the IPCC method we already selected above:

#### One at a time approach:

**Exercise:** Do the LCA for `wind`:

Do one "delta" LCA:

## 4) My third LCA - Multiple impact categories

Say we want to evaluate the indicator results for our randomAct for all EF v3 categories (with long-term emissions).

Simplest way: for loop, using `switch method`

## Revising my second and third LCA with `MultiLCA`

The `MultiLCA` allows the calculation of LCA results for multiple functional units and impact categories.  
One simply needs to create a `calculation setup`, i.e. a named set of functional units and LCIA methods.

Calculation setups: dictionary with lists of functional units and methods.

You can also create "fuller" DataFrames. Here is with code from [here](http://stackoverflow.com/questions/42984831/create-a-dataframe-from-multilca-results-in-brightway2): 

## Uncertainty

Values of exchanges in datasets may come with uncertainty.
See [documentation](https://stats-arrays.readthedocs.io/en/latest/) of `stats_array`.

Let's find out how uncertainty is defined in ecoinvent:

### Running a stochastic analysis (Monte Carlo)

Let's visualize the results:

## Let's now look at the import and export of data

Open the second notebook.